In [1]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from dateutil import parser

import mysql.connector
import json

import warnings
warnings.filterwarnings("ignore")

In [2]:
def get_deltas_channel( inputDF, list_of_channels ):

    dfage = inputDF[['DaysDelta', 'CreatedOn','ChannelID']]

    dfage['Age'] = pd.to_timedelta( dfage['DaysDelta'] - dfage['CreatedOn'] )/np.timedelta64(1,'D')
    dfage['Age'] = dfage['Age']/365.25

    dfage.drop(columns=['DaysDelta'], inplace=True)
    dfage.set_index('ChannelID', append=True, inplace=True)

    # To avoid duplication
    inputDF.drop(columns=['CreatedOn'], inplace=True)

    for i, channel in enumerate(list_of_channels):

        # Formatting DF
        helpDF = inputDF[ inputDF['ChannelID']==channel ].set_index(['ChannelID'], append=True)

        # Getting Deltas
        helpDF = helpDF - helpDF[['DaysDelta', 'views', 'subscribers', 'videoNr']].shift(1)
        # Fixing Dates delta format
        helpDF['DaysDelta'] = pd.to_timedelta(helpDF['DaysDelta'])/np.timedelta64(1,'D')

        # Loop counter
        if i==0:
            outDF = helpDF.copy() # First time, create df
        else:
            outDF = outDF.append( helpDF ) # any other, append

    # Adding the age by the momment of the extraction
    outDF = dfage.merge( outDF, left_index=True, right_index=True, how='right' )

    return outDF

In [3]:
def anonymizer(df, col_name, prefix='ANON_'):

    # Get unique values in column
    unique_values = df[col_name].unique()

    # Create a mapping dictionary of unique values to index strings
    mapping = {}
    for i, val in enumerate(unique_values):
        index_str = prefix + str(i)
        mapping[val] = index_str

    return mapping

In [4]:
# To store DB credentials
cred = json.load(open('DBCredentials.json', 'r'))

In [5]:
# To control anonimize
get_ann = True

## Channel IDs

In [6]:
# Connect to the database
cnx = mysql.connector.connect(
                                host=cred['host'],
                                database=cred['name'],
                                user=cred['user'],
                                password=cred['pass'])

query = \
            f"""
            SELECT 
                ChannelID, 
                ChannelName
            FROM 
                channel_details;
            """

# Reading channels
frChannels = pd.read_sql(query, cnx)

if get_ann:
    # Getting annonimizer names
    ann_ch_names = anonymizer(frChannels, 'ChannelName', 'Channel_')
    ann_ch_keys = anonymizer(frChannels, 'ChannelID', 'CH_ID_')
    # Applying
    frChannels = frChannels.assign( ChannelName = frChannels['ChannelName'].replace(ann_ch_names) )\
                           .assign( ChannelID = frChannels['ChannelID'].replace(ann_ch_keys) )

cnx.close()

frChannels.to_csv('Data/ChannelIDs.csv', index=False)

## Check evolution

In [7]:
def deltas( frame, feature ):
    """
    Extract the differences between lags for a determined feature.
    """
    
    fr = frame[['ExtractionDay', feature]].reset_index()
    fr = fr.pivot(index=['ExtractionDay'], columns=['ChannelName'])
    
    # Clean Column Names
    fr.columns = [ ch_name for (feat, ch_name) in fr.columns ]

    # Dict for variations
    variations = {}

    for chann in fr.columns:
        #print(chann)

        # Copy Time-Series
        variations[ chann ] = fr[[chann]].copy()

        # Shifting
        variations[ chann ]['Shift'] = variations[ chann ].shift()

        # Creating percentage diff
        variations[ chann ]['PercChange'] = variations[ chann ][chann].pct_change()

        # Check for NetDifferences in the timeline
        variations[ chann ]['NetDiff'] = variations[ chann ][chann] - variations[ chann ]['Shift']

        # Creating channel name column
        variations[ chann ]['YtChan'] = chann

        # Keeping only Variations
        variations[ chann ] = variations[ chann ][['YtChan', 'NetDiff', 'PercChange']]

    # Saving in unique dataframe
    fr_fin = pd.DataFrame()
    for K in variations.keys():
        fr_fin = fr_fin.append( variations[K] )

    return fr_fin


Channel Level

In [8]:
# Connect to the database
cnx = mysql.connector.connect(
                                host=cred['host'],
                                database=cred['name'],
                                user=cred['user'],
                                password=cred['pass'])

# Querying every atrribute and appending into a single DF
#dfch = pd.DataFrame(columns=['ChannelName', 'ExtractionDay', 'Attribute', 'Value'])

query = \
            f"""
            SELECT 
                channel_details.ChannelID, 
                ExtractionDay,
                ExtractionDay as DaysDelta,
                CreatedOn,
                channel_evolution.views,
                channel_evolution.subscribers,
                channel_evolution.videoNr
            FROM channel_evolution
            LEFT JOIN channel_details
            ON channel_evolution.ChannelID = channel_details.ChannelID;
            """

dfch = pd.read_sql(query, cnx)
dfch.set_index(['ExtractionDay'], inplace=True)

if get_ann:
    # Applying Anonimize
    dfch = dfch.assign( ChannelID = dfch['ChannelID'].replace(ann_ch_keys) )

cnx.close()

#dfch

In [9]:
dfchd = get_deltas_channel( dfch.copy(), dfch['ChannelID'].unique().tolist() )
#dfchd

In [10]:
dfch_act = dfch.drop(columns=['DaysDelta']).set_index(['ChannelID','CreatedOn'], append=True).stack().to_frame().rename(columns={0:'Actuals'}).copy()
#dfch_act

In [11]:
# Mergin both DFs
dfchd_act = dfchd.set_index('CreatedOn', append=True).stack().to_frame().rename(columns={0:'Evolution'})#.copy()
#dfchd_act

In [12]:
# Mergin attributes like subs, views and vids into a single table
dfchf = dfch_act.merge( dfchd_act, left_index=True, right_index=True, how='outer' )
dfchf.index.names = ['ExtractionDay', 'ChannelID', 'CreatedOn', 'Attribute']

#dfchf

In [13]:
# Mergin evolutions and actuals under a single column
dfchf = dfchf.stack().to_frame().rename(columns={0:'Values'})
dfchf.index.names = ['ExtractionDay', 'ChannelID', 'CreatedOn', 'Attribute', 'AnalysisType']
#dfchf

In [14]:
dfchf.to_csv('Data/ChannelLevel_OverTime.csv')#ann_ch_names

## Video Level

In [15]:
# video_statics | video_variables
# Connect to the database
cnx = mysql.connector.connect(
                                host=cred['host'],
                                database=cred['name'],
                                user=cred['user'],
                                password=cred['pass'])

query = \
            f"""
            SELECT
                video_statics.ChannelID,
                video_statics.video_id,
                video_statics.CreatedOn,
                video_statics.title,
                video_statics.duration/60 AS Minutes,
                video_statics.descr
            FROM
                video_statics
                ;
            """

frvids = pd.read_sql(query, cnx)

if get_ann:
    # Getting annonimizer dictionaries
    ann_video_keys = anonymizer(frvids, 'video_id', 'CH_ID_')
    ann_video_titles = anonymizer(frvids, 'title', 'video_title_')
    ann_video_descr = anonymizer(frvids, 'descr', 'video_descr_')

    # Applying
    frvids = frvids.pipe(lambda _df: _df.assign(**{ column : _df[column].replace(mapping) for column, mapping in zip(['video_id', 'title', 'descr', 'ChannelID'], [ann_video_keys, ann_video_titles, ann_video_descr, ann_ch_keys]) }) )

frvids.to_csv('Data/VideoIDs.csv', index=False, sep="\t")

cnx.close()

#frvids

### Video Evolution

In [16]:
# video_statics | video_variables
# Connect to the database
cnx = mysql.connector.connect(
                                host=cred['host'],
                                database=cred['name'],
                                user=cred['user'],
                                password=cred['pass'])

query = \
            f"""
            SELECT 
                video_statics.ChannelID,
                video_variables.video_id,
                video_variables.ExtractionDay,
                video_variables.viewCount,
                video_variables.likeCount,
                video_variables.commentCount,
                (DATEDIFF(video_variables.ExtractionDay, video_statics.CreatedOn)) AS VideoAge
            FROM 
                video_variables
            LEFT JOIN 
                video_statics
            ON 
                video_variables.video_id = video_statics.video_id
            ORDER BY 
                video_variables.ExtractionDay ASC, 
                video_statics.CreatedOn ASC;
            """
 
fr = pd.read_sql(query, cnx)

if get_ann:
    # Applying anonimization
    fr = fr.pipe(lambda _df: _df.assign(**{ column : _df[column].replace(mapping) for column, mapping in zip(['ChannelID', 'video_id'], [ann_ch_keys, ann_video_keys]) }) )

# Set index
fr = fr.set_index(['ChannelID', 'ExtractionDay', 'video_id'])

cnx.close()

#fr

In [17]:
videos_dfs = []

# Getting all unique videos IDs
all_vids_id = fr.index.get_level_values('video_id').unique()

for vid in all_vids_id:
    # Gets the details for the video level
    video_frame = fr.xs(vid, level=2)
    #print(vid)

    # Gets shift
    video_frame_shifted = video_frame - video_frame.shift(1)
    video_frame_shifted = video_frame_shifted.dropna() \
                                             .assign( video_id = vid ) \
                                             .set_index(['video_id'], append=True) \
                                             .stack() \
                                             .reset_index() \
                                             .rename(columns={'level_3':'Attribute', 0:'Evolution'}) \
                                             .set_index(['ChannelID', 'ExtractionDay', 'video_id', 'Attribute'])

    #Actuals
    video_frame = video_frame.assign( video_id = vid ) \
                             .set_index(['video_id'], append=True) \
                             .stack() \
                             .reset_index() \
                             .rename(columns={'level_3':'Attribute', 0:'Actuals'}) \
                             .set_index(['ChannelID', 'ExtractionDay', 'video_id', 'Attribute'])

    # Final Reporting
    df_vid = video_frame.merge( video_frame_shifted, left_index=True, right_index=True, how='left' ) \
                        .stack() \
                        .reset_index(level=4) \
                        .rename(columns={'level_4':'AnalysisType', 0:'Values'})

    # Append in list
    videos_dfs.append( df_vid )

dumpFr = pd.concat( videos_dfs, ignore_index=False )

In [ ]:
dumpFr.to_csv('Data/VideoLevel_OverTime.csv')